- [ ] não usar categorias abaixo do nível 2.
- retVal = Math.pow(rel, (1.0 - _cfg._divWeight)) * Math.pow(div, _cfg._divWeight);
- calculo do pr errado?? passando pelos vizinhos nao pelos log vizinhos????

In [2]:
import numpy as np
import pandas as pd
import recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
%matplotlib inline

# Category tree

In [3]:
df_categories=pd.read_json("../data/categories.json")

# dicionário alias title 2 way
dict_alias_title={}
for index, row in df_categories.iterrows():
    dict_alias_title[row['alias']]=row['title']
    dict_alias_title[row['title']]=row['alias']


category_tree= nx.DiGraph()
for index, row in df_categories.iterrows():
    if not row['parents']:
        category_tree.add_edge(row['alias'],'root') # root node if no parents
    else:
        for parent_label in row['parents']:
            category_tree.add_edge(row['alias'],parent_label)

undirected_category_tree=category_tree.to_undirected()
df_categories=None

## Remove lower levels than second in tree

# Load check-ins/reviews and treat the dataset

In [4]:
las_vegas_area=areamanager.delimiter_area(1)
df_business=pd.read_csv("../data/business.csv",usecols=['business_id','latitude','longitude','categories'])
df_business=areamanager.pois_in_area(las_vegas_area,df_business)
df_review=pd.read_csv("../data/review.csv",usecols=['business_id','user_id','date'])
df_review=pd.merge(df_review,df_business,on='business_id')
df_review['date']=pd.to_datetime(df_review['date'])
df_review.head()

Area selected: Las Vegas


,user_id,business_id,date,latitude,longitude,categories
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,2013-05-07 04:34:36,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."
1,RBXSJA372ilErzNwz0jXvQ,ujmEBvifdJM6h6RLv4wQIg,2017-02-09 06:45:18,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."
2,x3brMMbJrAW9PwW5A6YL5w,ujmEBvifdJM6h6RLv4wQIg,2018-08-14 05:25:33,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."
3,Skzdl0sWhW88525a1vr59g,ujmEBvifdJM6h6RLv4wQIg,2018-05-08 19:27:59,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."
4,3Y25VDfnQVcuc33T-U3Z6A,ujmEBvifdJM6h6RLv4wQIg,2014-10-11 06:57:18,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."


Filtragem dos POIs que foram visitados por cinco ou mais usuários

In [5]:
df_diff_users_visited=df_review[['user_id','business_id']].drop_duplicates().reset_index(drop=True).\
groupby('business_id').count().reset_index().rename(columns={"user_id":"diffusersvisited"})

df_diff_users_visited=df_diff_users_visited[df_diff_users_visited['diffusersvisited']>=5]
del df_diff_users_visited['diffusersvisited']
df_review=pd.merge(df_review,df_diff_users_visited,on='business_id',how='inner')
df_review.count()

user_id        878996
business_id    878996
date           878996
latitude       878996
longitude      878996
categories     878900
dtype: int64

Filtragem dos usuários que realizaram vinte ou mais check-ins

In [6]:
df_review['Count']=df_review.groupby(['user_id'])['user_id'].transform('count')
df_review=df_review[df_review['Count']>=20]
del df_review['Count']
df_review.count()

user_id        194427
business_id    194427
date           194427
latitude       194427
longitude      194427
categories     194416
dtype: int64

# Test and training set

In [7]:
df_review=df_review.sort_values(['user_id','date']).reset_index(drop=True)

In [8]:
ranks=df_review.groupby('user_id')['date'].rank(method='first')
ranks.head()

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: date, dtype: float64

In [9]:
#ranks=df_review.groupby('user_id')['date'].rank(method='first')
count=df_review['user_id'].map(df_review.groupby('user_id')['date'].apply(len))
count.head()

0    88
1    88
2    88
3    88
4    88
Name: user_id, dtype: int64

In [10]:
training_set_delimiter=(ranks/count)<0.8
test_set_delimiter=np.logical_not(training_set_delimiter)
training_set_delimiter

0          True
1          True
2          True
3          True
4          True
          ...  
194422    False
194423    False
194424    False
194425    False
194426    False
Length: 194427, dtype: bool

In [11]:
df_review_training=df_review[training_set_delimiter]
df_review_training.count()

user_id        152755
business_id    152755
date           152755
latitude       152755
longitude      152755
categories     152751
dtype: int64

In [12]:
df_review_test=df_review[test_set_delimiter]
df_review_test.count()

user_id        41672
business_id    41672
date           41672
latitude       41672
longitude      41672
categories     41665
dtype: int64

In [13]:
users=df_review_training['user_id'].drop_duplicates().reset_index(drop=True)
K = 10 # size of the recommendation list

In [65]:
VERY_SMALL_VALUE=-100.0

rec_list=pd.DataFrame(columns=['business_id','score','latitude','longitude','categories'])

# ILD

In [66]:
def category_dis_sim(category1,category2):
    dissim=0.0
    spd=nx.shortest_path_length(undirected_category_tree,category1,category2)
    sim = 1.0 / (1.0 + spd)
    dissim=1.0-sim
    return dissim
    

def min_dist_to_list_cat(business,rec_list):
    min_dissim=1.0
    rec_list_size=len(rec_list)
    if rec_list_size==0:
        min_dissim=1.0
    else:
        for index,row in rec_list.iterrows():
            local_min_distance=1
            cur_distance=0
            for category1 in eval(business['categories']):
                for category2 in eval(row['categories']):
                    cur_distance=category_dis_sim(
                        dict_alias_title[category1],
                        dict_alias_title[category2])
                    #print(category1,category2,cur_distance,local_min_distance)
                    local_min_distance=min(local_min_distance,cur_distance)
            min_dissim=min(min_dissim,local_min_distance)
    return min_dissim
        
def objective_ild(business,rec_list):
    # /*(1-_cfg._divWeight) * relevance + */_cfg._divWeight * diversity; where _cfg._divWeight = 0.9
    ret_val= 0.0
    diversity=0.0
    div_weight=0.9
    diversity=min_dist_to_list_cat(business,rec_list)
    ret_val = div_weight*diversity #+ (1-div_weight)*relevance
    return ret_val

# Genre Coverage

In [67]:
def relevant_categories_to_the_user(df_user_review):
    df=df_user_review
    # number of visits in pois
    df['Count']=df.groupby(['business_id','user_id'])['user_id'].transform('count')
    # merge to get mean poi visits, a column with Count and another with diffpoisvisited
    df_mean_poi_visits=pd.merge(\
    df[['user_id','Count']].groupby('user_id').sum().reset_index(level=0),\
    df[['user_id','business_id']].groupby('user_id').count().reset_index(level=0).rename(columns={"business_id":"diffpoisvisited"}),\
    on='user_id')
    
    df_mean_poi_visits['meanpoivisits']=df_mean_poi_visits['Count']/df_mean_poi_visits['diffpoisvisited']
    mean_poi_visits=df_mean_poi_visits['meanpoivisits'].iloc[0]
    # print(mean_poi_visits)
    
    # Relevant categories
    relevant_categories = set()    
    for index,row in df.iterrows():
        # Check if poi is relevant
        if row['Count'] > mean_poi_visits:
            # add relevant categories
            for category in eval(row['categories']):
                relevant_categories.add(category)
            #print(row['business_id'])
    return relevant_categories
    #print(relevant_pois)
    

def objective_genre_coverage(poi,rec_list,df_user_review):
    relevant_categories=relevant_categories_to_the_user(df_user_review)
    count_equal=0
    categories_in_rec_list=set()
    
    for index, row in rec_list.iterrows():
        for category in eval(row['categories']):
            categories_in_rec_list.add(category)
    
    for category in eval(poi['categories']):
        categories_in_rec_list.add(category)
    
    #print(categories_in_rec_list)
    
    for cat1 in relevant_categories:
        for cat2 in eval(poi['categories']):
            if cat1 == cat2:
                print(cat1)
                count_equal=count_equal+1
    return count_equal/len(relevant_categories)


# PR/Proportionality metric

In [68]:
def poi_distance_from_pois(poi,df_poi):
    return haversine(poi['latitude'],poi['longitude'],df_poi['latitude'],df_poi['longitude'])

def poi_neighbors(poi,df_poi,distance_km):
    return df_poi[poi_distance_from_pois(poi,df_poi)<distance_km]

def update_geo_cov(poi,df_user_review,rec_list_size,business_cover):
    user_log_size=len(df_user_review)
    neighbors=poi_neighbors(poi,df_user_review,0.1)
    num_neighbors=len(neighbors)
    vl=1
    COVER_OF_POI=user_log_size/rec_list_size
    accumulated_cover=0
    # Cover calc
    if num_neighbors<1:
        accumulated_cover=accumulated_cover+COVER_OF_POI
    else:
        cover_of_neighbor=COVER_OF_POI/num_neighbors
        for business_id in neighbors['business_id']:
            business_cover[business_id]=business_cover[business_id]+cover_of_neighbor
    accumulated_cover=accumulated_cover/user_log_size
    # end PR and DP
    DP=(accumulated_cover**2)/2
    
    for business_id in df_user_review['business_id']:
        if vl>=business_cover[business_id]:
            DP=DP+(vl-business_cover[business_id])**2    
    DP_IDEAL=user_log_size+0.5
    PR=1-DP/(DP_IDEAL)
    
    return PR




# GeoCat

In [69]:
def objective_ILD_GC_PR(poi,df_user_review,rec_list,rec_list_size,business_cover,current_proportionality,div_geo_cat_weight,div_weight):
    ild_div=objective_ild(poi,rec_list)
    gc_div=objective_genre_coverage(poi,rec_list,df_user_review)
    delta_proportionality=update_geo_cov(poi,df_user_review,rec_list_size,business_cover.copy())-current_proportionality
    if delta_proportionality<0:
        delta_proportionality=0
    div_cat = gc_div+ild_div/rec_list_size
    div_geo = delta_proportionality
    div=div_geo_cat_weight*div_geo+(1-div_geo_cat_weight)*div_cat
    return poi['score']**(1-div_weight)+div**div_weight

In [2]:

for user_id in users:
    tmp_rec_list=rec.mostpopular(df_review_training,K)
    tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
    print(tmp_rec_list)
    break
    # add other columns
    tmp_rec_list=pd.merge(tmp_rec_list,df_business,on='business_id').drop_duplicates().reset_index(drop=True)

    
    # User visits
    df_user_review=df_review_training[df_review_training['user_id']==user_id]
    #user_log_size=len(df_user_review)
    business_cover=dict()
    for business_id in df_user_review['business_id'].drop_duplicates().reset_index(drop=True):
        business_cover[business_id]=0
    # Constants
    #COVER_OF_POI=len(df_user_review)/len(tmp_rec_list)
    div_geo_cat_weight = 0.5 # beta,this is here because of the work to be done on parameter customization for each user
    div_weight = 0.5 # lambda
    current_proportionality=0
    break
    # post-processing for diversification
    for i in range(K):
        num_current_candidates=len(tmp_rec_list)
        poi_to_insert=None
        max_objective_value=VERY_SMALL_VALUE
        for index,poi_candidate in tmp_rec_list.iterrows():
            objective_value=objective_ILD_GC_PR(poi_to_insert,df_user_review,tmp_rec_list,K,\
                                                business_cover,current_proportionality,\
                                               div_geo_cat_weight,div_weight)
            
            if objective_value > max_objective_value:
                max_objective_value=objective_value
                poi_to_insert=poi_candidate

        if poi_to_insert is not None:
            rec_list=rec_list.append(poi_to_insert)
            # remove from tmp_rec_list
            tmp_rec_list=tmp_rec_list[tmp_rec_list['business_id']!=poi_to_insert['business_id']].reset_index(drop=True)
            
            current_proportionality=update_geo_cov(poi_to_insert,df_user_review,K,business_cover)
            
    
    print(user_id)
    break

NameError: name 'users' is not defined